# Power Outage Analysis

**Name(s)**: Walter Wong, Nathan Ko

**Website Link**: https://waltercywong.github.io/US_Power_Outages/

In [31]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# Introduction and Question Identification

This dataset has major power outage data in the continental U.S. from January 2000 to July 2016, provided by Laboratory for Advancing Sustainable Critical Infrastructure (LASCI)

Question: Are power outages worse during the summer? 

Null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.

Alternate hypothesis: The summer months have greater power outages than rest of the year.

TODO\
We use mean to do permutation test
Accounting for outliers of both sets, but outliers are often urban centers which are important




1534 rows, 54 columns

**Relevant columns:**
- `U.S._STATE`: Represents all the states in the continental U.S.
- `YEAR`: Indicates the year when the outage event occurred
- `MONTH`: Indicates the month when the outage event occurred
- `CUSTOMERS.AFFECTED`: Number of customers affected by the power outage event
- `CLIMATE.REGION`: U.S. Climate regions as specified by National Centers for Environmental Information (nine climatically consistent regions in continental U.S.A.)
- `DEMAND.LOSS.MW`: Amount of peak demand lost during an outage event (in Megawatt) [but in many cases, total demand is reported]
- `TOTAL.PRICE`: Average monthly electricity price in the U.S. state (cents/kilowatt-hour)
- `TOTAL.CUSTOMERS`: Annual number of total customers served in the U.S. state
- `POPULATION`: Population in the U.S. state in a year

In [39]:
outage_df = pd.read_excel('outage.xlsx')

# Data Cleaning

In the original data in the excel file, the 5 rows are unwanted as they are just the title and some notes about the data. The dataframe and its columns "start" from the 6th row downward. Additionally, below the columns, there is a row to specify the units of the column if necessary. We also removed this columns when cleaning the data as there are no actual observations.

At this point, the data being loaded has all the columns and observations in the excel file data. However, the columns ``OUTAGE.START.DATE`` and ``OUTAGE.START.TIME`` can be combined as the first always has a time of "00:00:00". The same is true for the restoration date and time. We combined these columns into ``OUTAGE.START`` and ``OUTAGE.RESTORATION``, dropping the original 4.

In [38]:
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned['OUTAGE.START'] = (outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned['OUTAGE.RESTORATION'] = (outage_cleaned['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [36]:
outage_cleaned.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,...,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00


# Univariate Analysis


## Power Outages by Region

In [67]:
df = outage_cleaned.copy()
df = df.rename(columns={'CLIMATE.REGION': 'Power Outage'})

df = df[['Power Outage']]
fig = px.histogram(df)
fig.update_layout(
    title_text="Power Outages by Region",
    xaxis_title="Climate Regions",
    yaxis_title="Number",
)
fig.show()

In [69]:
fig.write_html('plot_univar_1.html', include_plotlyjs='cdn')

### Explanation of Graph: 'Power Outages by Region'
This is a graph displaying the distributions of power outages within climate regions in the U.S. 

It is evident that the Northeast region has the highest number of power outages, totaling 350, while the West North Central region has the least, with fewer than 20 outages. For the most part, the other regions fall between 100 to 250 outages.




## Power Outages by Month

In [47]:
month_count = outage_cleaned.groupby('MONTH').count()

fig = px.bar(month_count, y = 'OBS')
fig.update_layout(
    title_text="Power Outages by Month",
    xaxis_title="Month",
    yaxis_title="Outage Count"
)
fig.show()

### Explanation of Graph: 'Power Outages by Month'
This is a graph displaying the distributions of power outages within each month.

Power outages are most prevalent during the summer, with a noticeable decease compared to the winter months. While there is a significant number of outages in winter, it doesn't compare to the frequency experienced during the summer season.







# Bivariate Analysis


## State Proportions of Customers

In [75]:
df = outage_cleaned.copy()
state_count = outage_cleaned.groupby('U.S._STATE')['POPULATION'].transform('mean')
df['pop_mean'] = state_count
state_means = df.groupby('U.S._STATE')['TOTAL.CUSTOMERS'].mean()/df.groupby('U.S._STATE').mean()['pop_mean']  
fig = px.bar(state_means)
fig.update_layout(
    title_text="State Proportion of Customers",
    xaxis_title="State",
    yaxis_title="Proportion of customers"
)
fig.show()

### Explanation of Graph: 'State Proportions of Customers'
This graph displays the proportion of customers to the population mean over the years of each state.

There isn't particularly any state that is a significant outlier. Hawaii and Maine have a small customer base while Maine has a surprising amount. 


# Interesting Aggregates	

## Mean Price vs Mean Customers per State

In [76]:
grouped_df = outage_cleaned.groupby('U.S._STATE').agg({'TOTAL.PRICE': 'mean', 'TOTAL.CUSTOMERS': 'mean'}).reset_index()
df = grouped_df.copy()
fig = px.scatter(df, x='TOTAL.PRICE', y='TOTAL.CUSTOMERS', text='U.S._STATE', title='Mean Price vs Mean Customers per State')
fig.show()

### Explanation of Graph: 'Mean Price vs Mean Customers per State'
'Mean Price vs Mean Customers per State' reveals the outliers of data set. Hawaii is outlandishly expensive despite have a small number of customers. California has a massively large customer base. 

# Assessment of Missingness


## NMAR Analysis
Column that we think is NMAR: ``CAUSE.CATEGORY.DETAIL``

``CAUSE.CATEGORY.DETAIL`` is a detailed description of the event categories causing the major power outages. It is NMAR because the cause doesnt fit in one of the limited categories due to being a multitude of causes. To possibly collect more data to make it MAR, we can have someone present at the site of the power outage and provide detailed observations of everything going on there. From this text, we can parse it for observations previously present in CAUSE.CATEGORY.DETAIL and count the number of unique observations we see. If the missing is correlated to higher counts of these occurrences, we can say that ``CAUSE.CATEGORY.DETAIL`` is MAR due to not being able to put a certain outage in a single category.


 

## MAR Analysis


**MAR Question:** Is ``CUSTOMERS.AFFECTED`` missing dependent on ``CLIMATE.CATEGORY``?

**Null hypothesis:** ``CUSTOMERS.AFFECTED`` is not missing dependent on ``CLIMATE.CATEGORY``.\
**Alternative hypothesis:** ``CUSTOMERS.AFFECTED`` is missing dependent on ``CLIMATE.CATEGORY``.

In [61]:
#MAR test that is not dependent on
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('CLIMATE.CATEGORY')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('CLIMATE.CATEGORY')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(10000):
    mar_test['CLIMATE.CATEGORY'] = np.random.permutation(mar_test['CLIMATE.CATEGORY'])
    perm_tvd = (mar_test.groupby('CLIMATE.CATEGORY')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('CLIMATE.CATEGORY')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
p_value = np.mean(tvds > observed_tvd)
p_value


/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91558/2577924923.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91558/2577924923.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



0.3601

Working with a significance level of 5% ($\alpha$ = 0.05), we cannot reject the null hypothesis because the p_value above is signifcantly higher than $\alpha$. As a result, we cannot make a conclusion on whether the missingness of ``CUSTOMERS.AFFECTED`` is dependent on ``CLIMATE.CATEGORY``.

**MAR Question:** Is ``CUSTOMERS.AFFECTED`` missing dependent on ``U.S._STATE``?

**Null hypothesis:** ``CUSTOMERS.AFFECTED`` is not missing dependent on ``U.S._STATE``.\
**Alternative hypothesis:** ``CUSTOMERS.AFFECTED`` is missing dependent on ``U.S._STATE``.

In [63]:
#MAR test that is dependent on
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(10000):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
p_value = np.mean(tvds > observed_tvd)
p_value

/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91558/2981418078.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/var/folders/75/4b4y2mlx2yv_34rkxb83ydkr0000gn/T/ipykernel_91558/2981418078.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



0.0

### Distributions of US States, Missing vs Not for CUSTOMERS.AFFECTED 

In [ ]:
mar_df = outage_cleaned.copy()
mar_df['MISSING'] = mar_df['CUSTOMERS.AFFECTED'].isna()
mar_df = mar_df.groupby(['U.S._STATE', 'MISSING'])['OBS'].count().reset_index()
px.bar(mar_df, x='U.S._STATE', y='OBS', color='MISSING', barmode="group")

### Empirical Distribution of Test Statistics

In [ ]:
fig = px.histogram(tvds)
if fig.data:
    fig.update_layout(
        shapes=[
            dict(
                type='line',
                x0=observed_tvd,
                x1=observed_tvd,
                y0=0,
                y1 = 1,
                line=dict(color='red', width=2)
            )
        ]
    )
fig.show()

Note: Red line is the observed test statistic.

### MAR Analysis Conclusion

Working with a significance level of 5% ($\alpha$ = 0.05), we can reject the null hypothesis because the p_value above is signifcantly lower than $\alpha$. As a result, we can conclude that the missingness of ``CUSTOMERS.AFFECTED`` is dependent on ``U.S._STATE``.

# Hypothesis Testing

**Question**: Are power outages worse during the summer compared to the rest of year?

To more specifically explore our question, we define the severity of power outages by how many customers are being affected and the summer consitutes of July, August, and September.

**Null Hypothesis:** The summer months of July, August, and September have the same average number of customers affected as the rest of the year.

**Alternative Hypothesis:** The summer months of July, August, and September have a greater average number of customers affected versus the rest of the year.

**Test Statistic:** The mean of ``CUSTOMERS.AFFECTED`` for the whole year subtracted from the mean of ``CUSTOMERS.AFFECTED`` in the summer months.

This is a good test statistic as a large test statistic means that there is a significance difference between the severity of outages in the summer vs the rest of the year. Under the null, this test statistic should also be zero. Additionally, since we are only finding if power outages are **worse** during the summer, we only need to compare if the simulated test statistics are larger than the observed.

**Significance Level:** 5% ($\alpha$ = 0.05)

In [65]:
observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_cleaned['CUSTOMERS.AFFECTED'].mean()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_copy['CUSTOMERS.AFFECTED'].mean()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

0.007650015253055972

### Hypothesis Testing Conclusion

Since the p-value above is significantly less than the significance level of 5%, we can reject the null hypothesis and conclude that it is likely power outages are worse during the summer compared to the rest of the year. This makes sense, as customers are more likely to turn on their AC during the hotter summer months, putting a greater strain on the electrical grid.

Number of missing values per column

In [ ]:
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned[outage_cleaned.isna().any(axis=1)]
outage_cleaned.T.isna().sum(axis=1).sort_values(ascending=False)


In [ ]:
outage_cleaned[outage_cleaned['IND.PRICE'].isna()]

Combined start date and time into one column 'start'

In [ ]:
# to delete
#  outage_cleaned['OUTAGE.START.TIME'][0] + outage_cleaned['OUTAGE.START.DATE'][0]
# outage_cleaned['OUTAGE.START.DATE'][0].date()
# outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: x.replace(hour=1, minute=10))
# outage_cleaned['OUTAGE.START.TIME'][0].hour

test_copy = outage_cleaned.copy()
# test_copy['OUTAGE.START.DATE'].dt.date + pd.to_timedelta(test_copy['OUTAGE.START.TIME'].dt.strftime('%H:%M:%S'))
test_copy['OUTAGE.START.TIME'][0].hour
# new_date = pd.datetime(test_copy['OUTAGE.START.DATE'][0].year, test_copy['OUTAGE.START.DATE'][0].month, test_copy['OUTAGE.START.DATE'][0].day, test_copy['OUTAGE.START.TIME'][0].hour, test_copy['OUTAGE.START.TIME'][0].minute, 0)

# test_copy['test_date'] = test_copy.apply(lambda x, y: pd.datetime(x.year, x.month, x.date, y.hour, y.minute, 0), test_copy['OUTAGE.START.DATE'], test_copy['OUTAGE.START.TIME'])
# pd.to_datetime(test_copy['OUTAGE.START.TIME'][0])
# datetime_object = datetime.combine(datetime.date.today(), time)

pd.datetime.combine(pd.datetime.today().date(), test_copy['OUTAGE.START.TIME'][0])
test_copy['OUTAGE.START.TIME'] = test_copy['OUTAGE.START.TIME'].apply(lambda x: pd.datetime.combine(pd.datetime.today().date(), x) if type(x) != float else pd.datetime(2023, 1, 1, 0, 0, 0))
test_copy['OUTAGE.START.TIME'][0]
test_copy['OUTAGE.START.DATE'].dt.date + pd.to_timedelta(test_copy['OUTAGE.START.TIME'])#.dt.strftime('%H:%M:%S'))


Combined restoration date and time into one column 'restoration start'

In [ ]:
# to delete
test_copy = outage_cleaned.copy()
# test_copy['OUTAGE.START.DATE'][0].hour = 1
# test_copy['test_date'] = test_copy.apply(lambda x: pd.datetime(x.loc['OUTAGE.START.DATE'].year, x.loc['OUTAGE.START.DATE'].month, x.loc['OUTAGE.START.DATE'].date, x.loc['OUTAGE.START.TIME'].hour, x.loc['OUTAGE.START.TIME'].minute, 0))
# pd.datetime.combine(test_copy['OUTAGE.START.DATE'][0].date(), test_copy['OUTAGE.START.TIME'][0])
# test_copy['test_date'] = test_copy.apply(lambda x: True if x['U.S._STATE']=='Minnesota' else False)
# test_copy
# test_copy['OUTAGE.START.DATE'][0] = pd.to_datetime(str(test_copy['OUTAGE.START.DATE'][0]).split(' ')[0] + ' ' + str(test_copy['OUTAGE.START.TIME'][0]))

test_copy['OUTAGE.START'] = (test_copy['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  test_copy['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
test_copy['OUTAGE.RESTORATION'] = (test_copy['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  test_copy['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
test_copy = test_copy.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [ ]:
# type(test_copy.iloc[-1]['OUTAGE.START.DATE'])
test_copy.tail(5)

In [ ]:
outage_cleaned

In [ ]:
pd.set_option('max_column', None)
# outage_cleaned[outage_cleaned['IND.PRICE'].isna()]
# outage_cleaned[outage_cleaned['OUTAGE.DURATION'].isna()]

 MAR: state vs customers missing

 NMAR: demand loss, cause category detail
 
 just do cause category, missing because cannot properly fit into 

In [ ]:
#MAR test
#take states
#make columns missing=True and missing=False for customers_affected missing or not
#groupby, take counts of true/false columns fuck pivot table
mar_test = outage_cleaned.copy()
mar_test['missingTrue'] = mar_test['CUSTOMERS.AFFECTED'].isna()
mar_test['missingFalse'] = mar_test['CUSTOMERS.AFFECTED'].notna()
observed_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2

tvds = []
for _ in range(100):
    mar_test['U.S._STATE'] = np.random.permutation(mar_test['U.S._STATE'])
    perm_tvd = (mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum() / mar_test.groupby('U.S._STATE')['missingTrue', 'missingFalse'].sum().sum()).T.diff().iloc[-1].abs().sum()/2
    tvds.append(perm_tvd)
np.mean(tvds > observed_tvd)
# tvds

Find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months

Then shuffle 'month' column 1000 times, find the difference, find percentage of those that had higher difference than observed

In [ ]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed
outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median()

In [ ]:
observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median() - outage_cleaned['CUSTOMERS.AFFECTED'].median()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].median() - outage_copy['CUSTOMERS.AFFECTED'].median()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

In [ ]:
#question: does month have an effect on the severity of power outages?
#null hypothesis: The summer months (7, 8, 9) have the same average number of customers affected as the rest of the year.
#alternate hypothesis: the summer months have greater power outages than rest of the year
#note: use mean to do permutation test
#accounting for outliers of both sets, but outliers are often urban centers which are important

#find observed statistic, which is mean of customers affected of 7,8,9 minus mean of all months
# shuffle month column 1000 times, find diff, find percentage of those that had higher diff than observed

observed_diff = outage_cleaned[(outage_cleaned['MONTH'] == 7) | (outage_cleaned['MONTH'] == 8) | (outage_cleaned['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_cleaned['CUSTOMERS.AFFECTED'].mean()

outage_copy = outage_cleaned.copy()
p_vals = []
diffs_list = []
for __ in range(100):
    for _ in range(1000):
        outage_copy['MONTH'] = np.random.permutation(outage_copy['MONTH'])
        perm_diff = outage_copy[(outage_copy['MONTH'] == 7) | (outage_copy['MONTH'] == 8) | (outage_copy['MONTH'] == 9)]['CUSTOMERS.AFFECTED'].mean() - outage_copy['CUSTOMERS.AFFECTED'].mean()
        diffs_list.append(perm_diff)
    p_vals.append(np.mean(diffs_list > observed_diff))
np.mean(p_vals)

### Cleaning and EDA

### Assessment of Missingness

In [ ]:
# TODO

In [ ]:
px.bar(outage_cleaned[outage_cleaned['CUSTOMERS.AFFECTED'].isna()].groupby('U.S._STATE')['OBS'].count())

In [ ]:
px.bar(outage_cleaned[outage_cleaned['CUSTOMERS.AFFECTED'].notna()].groupby('U.S._STATE')['OBS'].count())

### Hypothesis Testing

In [ ]:
# the time of the year has no effect on the severity of power outages (how many customers affected)